In [1]:
import pandas as pd
import time
import numpy as np
#from fasthaversine import haversine

In [2]:
start = time.time()
events_df = pd.read_csv("eventi.csv")
eventi_gestiti_df = pd.read_csv("eventi_gestiti.csv")
eventi_df = pd.concat([events_df, eventi_gestiti_df])
eventi_df["city"] = 0
df_cities = pd.read_csv("european_cities_us_standard.csv")
df_cities.columns = ["city", "ISO", "lat", "lon"]
df_cities["lat"] = pd.to_numeric(df_cities["lat"])
df_cities["lon"] = pd.to_numeric(df_cities["lon"])
end = time.time()
print(f"Time elapsed: {(end - start)/60} min")

KeyboardInterrupt: 

In [ ]:
start = time.time()
eventi_df['IdCausaEvento'] = eventi_df['IdCausaEvento'].replace([0., 1., 3., 4., 5., 7., 9., 12., 13., 15., 16., 19.],0)
eventi_df['IdCausaEvento'] = eventi_df['IdCausaEvento'].replace([2.],1)
eventi_df = eventi_df.dropna(subset=['IdCausaEvento', 'Lat', 'Lon'])
eventi_df = eventi_df.drop(eventi_df[(eventi_df["Lat"] <= 0) & (eventi_df["Lon"] <= 0)].index)
eventi_df.drop(columns=['Unnamed: 0', 'IdEvento', 'IdUtente', 'IdEvento.1', 'IDSecuritas', 'idMetatrak', 'idTelic', 'idRaliacom', 'idEnigma', 'IdNota', 'IdUtenteLast', 'DataPresaInCarico', 'SN_Movimento', 'IDAlarmFlat', 'IDTaigete', 'IDKFT', 'IDMobotix', 'IDHesagard', 'IDVideotecnologie', 'IDEpsaEvents', 'IDEnfora', 'IDXtraX'], inplace=True)
eventi_df["lat"] = pd.to_numeric(eventi_df["Lat"])
eventi_df["lon"] = pd.to_numeric(eventi_df["Lon"])
end = time.time()
print(f"Time elapsed: {(end - start)/60} min")

In [ ]:
def dist_between_two_lat_lon(*args):
    from math import asin, cos, radians, sin, sqrt
    lat1, lat2, long1, long2 = map(radians, args)

    dist_lats = abs(lat2 - lat1) 
    dist_longs = abs(long2 - long1) 
    a = sin(dist_lats/2)**2 + cos(lat1) * cos(lat2) * sin(dist_longs/2)**2
    c = asin(sqrt(a)) * 2
    radius_earth = 6378 # the "Earth radius" R varies from 6356.752 km at the poles to 6378.137 km at the equator.
    return c * radius_earth

In [ ]:
def find_closest_lat_lon(data, v):
    try:
        return min(city_list, key=lambda p: dist_between_two_lat_lon(v['lat'],p['lat'],v['lon'],p['lon']))
    except TypeError:
        print('Not a list or not a number.')

In [ ]:
def find_city(row):
    return find_closest_lat_lon(city_list, {"lon":row["Lon"], "lat":row["Lat"]})

In [ ]:
city_list = df_cities[["lon", "lat", "city"]].to_dict("records")
#eventi_df["city_dict"] = eventi_df[["lon", "lat"]].to_dict("records")

In [ ]:
sample_df = eventi_df.sample(frac=0.005)

In [ ]:
start = time.time()
cities = sample_df.apply(find_city, axis = 1)
end = time.time()
print(f"Time elapsed: {(end - start)/60} min")

In [ ]:
cities_list = []

In [ ]:
for el in cities:
    cities_list.append(el.get("city"))

In [ ]:
sample_df["city"] = cities_list

In [ ]:
sample_df.to_csv("eventi_cities_2.csv")

In [ ]:
#sample_df["ratio"] = find_closest_lat_lon(city_list, {"lon": sample_df["Lon"], "lat":sample_df["Lat"]})